<a href="https://colab.research.google.com/github/hanhluukim/replication-topic-modelling-in-embedding-space/blob/main/notebook_replication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!git clone https://github.com/hanhluukim/replication-topic-modelling-in-embedding-space.git

In [2]:
cd /content/replication-topic-modelling-in-embedding-space

/content/replication-topic-modelling-in-embedding-space


In [3]:
#!pip install -r "/content/replication-topic-modelling-in-embedding-space/requirements.txt"

# **Datensatz, Vorverarbeitung und BOW-Repräsentationen**
1. Vocabular erstellen
2. BOW-Repräsentationen für allen Teildatensätzen

In [4]:
from src.preprare_dataset import TextDataLoader
import pandas as pd

In [5]:
# init TextDataLoader für die Datenquelle 20 News Groups
# Daten abrufen vom Sklearn, tokenisieren und besondere Charaktern entfernen
textsloader = TextDataLoader(source="20newsgroups", train_size=None, test_size=None)
textsloader.load_tokenize_texts("20newsgroups")

loading texts: ...
finished load!


In [6]:
# Beispiel von Textdaten
textsloader.show_example_raw_texts(n_docs=2)

check some sample texts of the dataset
['From', ':', 'lerxst', '@', 'wam', '.', 'umd', '.', 'edu', '(', "where's", 'my', 'thing', ')', 'Subject', ':', 'WHAT', 'car', 'is', 'this', '!', '?', 'Nntp', 'Posting', 'Host', ':', 'rac3', '.', 'wam', '.', 'umd', '.', 'edu', 'Organization', ':', 'University', 'of', 'Maryland', ',', 'College', 'Park', 'Lines', ':', '15', 'I', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'I', 'saw', 'the', 'other', 'day', '.', 'It', 'was', 'a', '2', 'door', 'sports', 'car', ',', 'looked', 'to', 'be', 'from', 'the', 'late', '60s', '/', 'early', '70s', '.', 'It', 'was', 'called', 'a', 'Bricklin', '.', 'The', 'doors', 'were', 'really', 'small', '.', 'In', 'addition', ',', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', '.', 'This', 'is', 'all', 'I', 'know', '.', 'If', 'anyone', 'can', 'tellme', 'a', 'model', 'name', ',', 'engine', 'specs', ',', 'years', 'of', 'production'

In [7]:
# Vorverarbeitung von Daten mit folgenden Schritten:
textsloader.preprocess_texts(length_one_remove=True, punctuation_lower = True, stopwords_filter = True)

start: preprocessing: ...
finised: preprocessing!


In [8]:
# Daten zerlegen für Train, Test und Validation. Erstellen Vocabular aus dem Trainset
textsloader.split_and_create_voca_from_trainset(max_df=0.7, min_df=10, stopwords_remove_from_voca=True)

permuted indices for the train set: [ 92 194  88 222 122 132  28 109 153 133 212  72 231 220 188]
start creating vocabulary ...
length of the vocabulary: 348
sample ten words of the vocabulary: ['due', 'sale', 'told', 'tin', 'stop', 'article', 'put', 'religious', 'bill', 'texas']
length word2id list: 348
length id2word list: 348
finished: creating vocabulary


In [9]:
# Erstellen BOW-Repräsentation für ETM Modell
for_lda_model = False 
word2id, id2word, train, test, val = textsloader.create_bow_and_savebow_for_each_set(for_lda_model=for_lda_model)

length train-documents-indices : 4526
length of the vocabulary: 348


start: creating bow representation...
top 10 - word-id of the doc: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
max word-id: 347
min word-id: 0
max doc-id: 149
min doc-id: 0
all docs: 4526
all words: 4526
docidx unique 150
words unique: 348
ndocs: 150
vocab-size: 348
finised creating bow input!

start: creating bow representation...
top 10 - word-id of the doc: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
max word-id: 347
min word-id: 0
max doc-id: 49
min doc-id: 0
all docs: 1465
all words: 1465
docidx unique 50
words unique: 325
ndocs: 50
vocab-size: 348
finised creating bow input!

start: creating bow representation...
top 10 - word-id of the doc: [0, 1, 3, 4, 5, 6, 7, 9, 16, 17]
max word-id: 347
min word-id: 0
max doc-id: 49
min doc-id: 0
all docs: 719
all words: 719
docidx unique 50
words unique: 240
ndocs: 50
vocab-size: 348
finised creating bow input!

start: creating bow representation...
top 10 - word-id of the doc: [2, 5, 6, 7, 8, 9, 1

# **Anzeigen von dem Vocabular und IDs**



In [10]:
# show for samples: 100 word2id and id2 word
word2id_df_100 = pd.DataFrame()
word2id_df_100['word'] = list(word2id.keys())[:100]
word2id_df_100['id'] = list(word2id.values())[:100]
word2id_df_100

,word,id
0,due,0
1,sale,1
2,told,2
3,tin,3
4,stop,4
...,...,...
95,hell,95
96,truth,96
97,toronto,97
98,figure,98


In [11]:
# Kontrollieren die Größen von verschiedenen Datensätzen
print(f'Size of the vocabulary after prprocessing ist: {len(textsloader.vocabulary)}')
print(f'Size of train set: {len(train["tokens"])}')
print(f'Size of val set: {len(val["tokens"])}')
print(f'Size of test set: {len(test["test"]["tokens"])}')

Size of the vocabulary after prprocessing ist: 348
Size of train set: 150
Size of val set: 100
Size of test set: 50


# **Wieder erstellen von Dokumenten nach der Vorverarbeitung**

In [12]:
# re-erstellen von Dokumenten nach der Vorverarbeitungen. Die Dokumenten sind in Wörtern und werden für Word-Embedding Training benutzt
docs_tr, docs_t, docs_v = textsloader.get_docs_in_words_for_each_set()
train_docs_df = pd.DataFrame()
train_docs_df['text-after-preprocessing'] = [' '.join(doc) for doc in docs_tr[:100]]
train_docs_df

,text-after-preprocessing
0,reply state message apr nntp posting host univ...
1,university university internet
2,nntp posting host article writes results days ...
3,nntp posting host cs system phone computer means
4,reply college disclaimer care info robert writ...
...,...
95,question usa distribution sci article world wo...
96,nntp posting host newsreader tin version de wr...
97,state university idea find related information...
98,originator nntp posting host research article ...


# **Word-Embedding-Trainieren aus dem Train-documents**

In [13]:
from src.embedding import WordEmbeddingCreator
from pathlib import Path

save_path = Path.joinpath(Path.cwd(), "vocab_embedding.txt")
wb_creator = WordEmbeddingCreator(model_name="cbow", documents = docs_tr, save_path= save_path)
wb_creator.train(min_count=2, embedding_size= 10)
vocab = list(word2id.keys())
wb_creator.create_and_save_vocab_embedding(vocab, save_path)

word-embedding train begins
word-embedding train finished
length of vocabulary from word-embedding model 348
length of the vocabulary of prepraring-dataset-vocabulary: 348


100%|██████████| 348/348 [00:00<00:00, 47933.33it/s]


True